In [7]:
#!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:


import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import torch
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)

nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    tokens = word_tokenize(text)

    tokens = [token.lower() for token in tokens if token not in punctuations]

    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    tokens = [token for token in tokens if token not in stop_words]

    processed_text = ' '.join(tokens)

    return processed_text
#df = pd.read_csv('cyber-threat-intelligence_all.csv')
train_data = pd.read_csv('cyber-threat-intelligence-splited_train.csv')
valid_data = pd.read_csv('cyber-threat-intelligence-splited_validate.csv')
test_data = pd.read_csv('cyber-threat-intelligence-splited_test.csv')

train_data=train_data.drop(labels='Unnamed: 0',axis=1)
train_data['label'].fillna(value='benign', inplace=True)
train_data=train_data.fillna(0)

valid_data=valid_data.drop(labels='Unnamed: 0',axis=1)
valid_data['label'].fillna(value='benign', inplace=True)
valid_data=valid_data.fillna(0)

test_data=test_data.drop(labels='Unnamed: 0',axis=1)
test_data['label'].fillna(value='benign', inplace=True)
test_data=test_data.fillna(0)

train_df = train_data[['text','label']]
valid_df = valid_data[['text','label']]
test_df = test_data[['text','label']]

train_df['text_new'] = train_df['text'].apply(preprocess_text)
valid_df['text_new'] = valid_df['text'].apply(preprocess_text)
test_df['text_new'] = test_df['text'].apply(preprocess_text)


train_df = train_df.drop(labels=['text'],axis=1)
train_df['text_new'] = train_df['text_new'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

valid_df = valid_df.drop(labels=['text'],axis=1)
valid_df['text_new'] = valid_df['text_new'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

test_df = test_df.drop(labels=['text'],axis=1)
test_df['text_new'] = test_df['text_new'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)


for df in [train_df, valid_df, test_df]:
    df['label'].loc[
        (df['label']=="MD5") |
        (df['label']=="REGISTRYKEY") |
        (df['label']=="EMAIL") |
        (df['label']=="Infrastucture") |
        (df['label']=="DOMAIN") |
        (df['label']=="SHA1") |
        (df['label']=="IPV4") |
        (df['label']=="campaign") |
        (df['label']=="URL") |
        (df['label']=="SHA2") |
        (df['label']=="vulnerability") |
        (df['label']=="FILEPATH") |
        (df['label']=="tools") |
        (df['label']=="TIME") |
        (df['label']=="url") |
        (df['label']=="hash")
    ] = "NEED_ATTENTION"


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_SEQ_LENGTH = 128

def encode_texts(texts, max_length):
    return tokenizer(
        texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt'
    )

train_encodings = encode_texts(train_df['text_new'], max_length=MAX_SEQ_LENGTH)
valid_encodings = encode_texts(valid_df['text_new'], max_length=MAX_SEQ_LENGTH)
test_encodings = encode_texts(test_df['text_new'], max_length=MAX_SEQ_LENGTH)


# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=Y.shape[1])

lb = LabelBinarizer()
train_labels = lb.fit_transform(train_df['label'])
valid_labels = lb.transform(valid_df['label'])
test_labels = lb.transform(test_df['label'])
train_labels = torch.tensor(train_labels, dtype=torch.float32)
valid_labels = torch.tensor(valid_labels, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
valid_dataset = CustomDataset(valid_encodings, valid_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=train_labels.shape[1])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


def compute_metrics(p):
    pred, labels = p
    pred = (pred > 0.5).astype(int)
    accuracy = (pred == labels).mean()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-1-f0a5c0f8dbb2>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text_new'] = train_df['text'].apply(preprocess_text)
<ipython-input-1-f0a5c0f8dbb2>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

Epoch,Training Loss,Validation Loss,Accuracy
1,0.259800,0.216636,0.916267
2,0.219500,0.185357,0.932106
3,0.138500,0.177824,0.937757


<ipython-input-1-f0a5c0f8dbb2>:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-1-f0a5c0f8dbb2>:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-1-f0a5c0f8dbb2>:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [3]:
# train_preds = trainer.predict(train_dataset)
# train_accuracy = (train_preds.predictions.flatten() > 0.5).astype(int) == train_preds.label_ids
# print(f"Training Accuracy: {train_accuracy.mean()}")


model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

val_results = trainer.evaluate()
print(f"Validation Results: {val_results}")
print(f"Validation Accuracy: {val_results['eval_accuracy']}")

test_results = trainer.evaluate(test_dataset)
print(f"Test Results: {test_results}")
print(f"Test Accuracy: {test_results['eval_accuracy']}")

<ipython-input-1-f0a5c0f8dbb2>:139: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Validation Results: {'eval_loss': 0.17782370746135712, 'eval_accuracy': 0.9377568493150685, 'eval_runtime': 11.2071, 'eval_samples_per_second': 130.275, 'eval_steps_per_second': 16.329, 'epoch': 3.0}
Validation Accuracy: 0.9377568493150685
Test Results: {'eval_loss': 0.17329873144626617, 'eval_accuracy': 0.9375856164383561, 'eval_runtime': 11.3047, 'eval_samples_per_second': 129.15, 'eval_steps_per_second': 16.188, 'epoch': 3.0}
Test Accuracy: 0.9375856164383561


In [14]:
def predict(text, model, tokenizer, device, class_labels):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_SEQ_LENGTH)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    model.to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.sigmoid(logits).detach().cpu().numpy()

    if probs.shape[1] == 1:
        pred = probs.flatten()
        if pred >= 0.5:
            return "NEED_ATTENTION", None
        else:
            return "Other", None
    else:
        subclass_pred = np.argmax(probs, axis=1)
        subclass_label = class_labels[subclass_pred[0]]
        return "NEED_ATTENTION", subclass_label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_text = "send notification use telegram api sendmessage function let anyone use telegram bot send message public channel"

class_labels = lb.classes_

pred_class, subclass_label = predict(sample_text, model, tokenizer, device, class_labels)
print(f"Prediction for sample text: {pred_class}")
if subclass_label:
    print(f"Predicted subclass: {subclass_label}")


Prediction for sample text: NEED_ATTENTION
Predicted subclass: benign


In [11]:
test_df['text_new'][4]

'send notification use telegram api sendmessage function let anyone use telegram bot send message public channel'

In [17]:
class_labels = [
    "MD5", "REGISTRYKEY", "EMAIL", "Infrastucture", "DOMAIN",
    "SHA1", "IPV4", "campaign", "URL", "SHA2", "vulnerability",
    "FILEPATH", "tools", "TIME", "url", "hash", "NEED_ATTENTION"
]

def predict(text, model, tokenizer, device, class_labels):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_SEQ_LENGTH)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    model.to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).detach().cpu().numpy()

    subclass_pred = np.argmax(probs, axis=1)
    subclass_label = class_labels[subclass_pred[0]]

    print(f"Probabilities: {probs}")
    print(f"Predicted class index: {subclass_pred[0]}")

    return "NEED_ATTENTION", subclass_label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_text = "send notification use telegram api sendmessage function let anyone use telegram bot send message public channel"

pred_class, subclass_label = predict(sample_text, model, tokenizer, device, class_labels)
print(f"Prediction for sample text: {pred_class}")
print(f"Predicted subclass: {subclass_label}")


Probabilities: [[2.0381644e-04 1.3678579e-04 1.8570552e-04 9.9906415e-01 8.7822606e-05
  1.3364466e-04 1.2356021e-04 6.4559070e-05]]
Predicted class index: 3
Prediction for sample text: NEED_ATTENTION
Predicted subclass: Infrastucture


In [18]:
print(f"Number of unique labels: {train_labels.shape[1]}")

Number of unique labels: 8
